In [1]:
import numpy as np
import pandas as pd

In [77]:
pois = pd.read_csv('../data/201_POI_intersect.csv', index_col=[0])
pois.head()

,FULLNAME,RTTYP,MTFCC,FNAME2,LINEARID_2,FULLNAME_2,RTTYP_2,MTFCC_2,FNAME2_2,LINEARID_2_2,...,Latitude,Longitude,Altitude,Speed(m/s),Bearing,Accuracy,WKT_path,wkt_poi,POI_distance,Shot_angle
LINEARID,,,,,,,,,,,,,,,,,,,,,
110739489642,Bienville Blvd,M,S1400,Bienville Blvd,110739489642,Bienville Blvd,M,S1400,Bienville Blvd,1104354855431,...,30.248831,-88.191650,-27,0.00,0.0,16,Point (533435.97925063 27784.58364286),"Polygon ((533463.43976244 27776.68388771, 5334...",14.754,84.760
110739489642,Bienville Blvd,M,S1400,Bienville Blvd,110739489642,Bienville Blvd,M,S1400,Bienville Blvd,1104354855431,...,30.248812,-88.191597,-27,5.64,115.2,16,Point (533441.10625579 27782.43802815),"Polygon ((533463.43976244 27776.68388771, 5334...",9.322,84.760
110739489642,Bienville Blvd,M,S1400,Bienville Blvd,110739489642,Bienville Blvd,M,S1400,Bienville Blvd,1104354855431,...,30.248802,-88.191513,-29,6.74,107.0,16,Point (533449.17675619 27781.33173657),"Polygon ((533463.43976244 27776.68388771, 5334...",4.706,84.760
110739489642,Bienville Blvd,M,S1400,Bienville Blvd,110739489642,Bienville Blvd,M,S1400,Bienville Blvd,1104354855431,...,30.248783,-88.191437,-28,0.00,0.0,16,Point (533456.50656458 27779.17273588),"Polygon ((533463.43976244 27776.68388771, 5334...",8.442,84.760
110739489642,Bienville Blvd,M,S1400,Bienville Blvd,110739498140,Westward Ho,M,S1400,Westward Ho,110739489642,...,30.248762,-88.191109,-29,0.00,0.0,21,Point (533488.06588666 27776.6550353),"Polygon ((533513.41763944 27781.26564515, 5335...",11.332,84.771


# Camera feature

In [65]:
import cv2
import matplotlib.pyplot as plt 
from scipy.interpolate import griddata

def camera(picture, outputshape=(400, 400), theta=60, phi=-90, windowsize=(55, 55)):
    
    imgsize=picture.shape[0]
    orix, oriy = picture.shape[0]/2, picture.shape[1]/2
    
    map1 = []
    map2 = []
    
    #print(orix)
    
    # Mapping construction DESTINATION
    for x in range(outputshape[0]):
        for y in range(outputshape[1]):
            map2.append([x,y])
    #print(np.array(map2).shape)
    
    for p in np.linspace(phi-windowsize[1]/2, phi+windowsize[1]/2, outputshape[1]):
        for r in np.linspace(theta-windowsize[0]/2, theta+windowsize[0]/2, outputshape[0]):
            p1 = orix + (r/90*orix) * np.cos(p/180*np.pi)
            p2 = oriy + (r/90*orix) * np.sin(p/180*np.pi)
            map1.append([int(p1), int(p2)])

        
    grid_x, grid_y = np.mgrid[0:outputshape[0]:outputshape[0]*1j, 0:outputshape[1]:outputshape[0]*1j]
    grid_z = griddata(map2, map1, (grid_x, grid_y), method='cubic')
    map_x = np.append([], [ar[:,0] for ar in grid_z]).reshape(outputshape)
    map_y = np.append([], [ar[:,1] for ar in grid_z]).reshape(outputshape)
    map_x_32 = map_x.astype('float32')
    map_y_32 = map_y.astype('float32')
    
    trf = cv2.remap(picture,
                     map_x_32, map_y_32, 
                     cv2.INTER_LINEAR,)
    
    rows,cols = trf.shape[:2]

    M = cv2.getRotationMatrix2D((cols/2,rows/2),-90,1)
    dst = cv2.warpAffine(trf,M,(cols,rows))
    return dst

In [66]:
pois.columns

Index(['FULLNAME', 'RTTYP', 'MTFCC', 'FNAME2', 'LINEARID_2', 'FULLNAME_2',
       'RTTYP_2', 'MTFCC_2', 'FNAME2_2', 'LINEARID_2_2', 'FULLNAME_2_2',
       'RTTYP_2_2', 'MTFCC_2_2', 'FNAME2_2_2', 'WKT_intersection', 'FID',
       'Type', 'Mode', 'Timestamp(', 'Latitude', 'Longitude', 'Altitude',
       'Speed(m/s)', 'Bearing', 'Accuracy', 'WKT_path', 'wkt_poi',
       'POI_distance', 'Shot_angle'],
      dtype='object')

In [67]:
subset = pois[pois['FID']== 101]
subset = pois[['FID', 'Shot_angle', 'Bearing', 'Timestamp(', 'POI_distance']]
subset.sort_values('Timestamp(')
subset.head()

,FID,Shot_angle,Bearing,Timestamp(,POI_distance
LINEARID,,,,,
110739489642,87,84.752,0.0,11.015,6.814
110739489642,73,84.771,0.0,15.099,2.737
110739489642,88,264.771,0.0,15.099,2.723
110739489642,74,264.674,0.0,19.149,2.395
110739489642,59,84.709,0.0,21.177,2.851


In [68]:
def parallel_scoring(df):
    sin1 = np.sin(np.radians(df.Shot_angle))
    sin2 = np.sin(np.radians(df.Bearing))
    cos1 = np.cos(np.radians(df.Shot_angle))
    cos2 = np.cos(np.radians(df.Bearing))
    return np.sqrt((sin1 - sin2)**2 + (cos1 - cos2)**2)/2

subset['pscore'] = subset.apply(parallel_scoring, axis=1, )
subset[['FID', 'Shot_angle', 'Bearing', 'Timestamp(', 'pscore']].sort_values('pscore').head()

/home/student/anaconda3/envs/tensor/lib/python3.5/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,FID,Shot_angle,Bearing,Timestamp(,pscore
LINEARID,,,,,
110739492146,23,86.835,86.8,462.793,0.000305
110739500916,53,359.160,359.1,851.786,0.000524
110740795643,9,359.085,359.0,846.719,0.000742
110739489642,93,83.220,83.0,74.875,0.001920
110739500916,53,359.160,359.4,852.797,0.002094


In [69]:
subset.head()

,FID,Shot_angle,Bearing,Timestamp(,POI_distance,pscore
LINEARID,,,,,,
110739489642,87,84.752,0.0,11.015,6.814,0.673993
110739489642,73,84.771,0.0,15.099,2.737,0.674115
110739489642,88,264.771,0.0,15.099,2.723,0.738626
110739489642,74,264.674,0.0,19.149,2.395,0.739196
110739489642,59,84.709,0.0,21.177,2.851,0.673716


In [70]:
subset.groupby('FID').min().head()

,Shot_angle,Bearing,Timestamp(,POI_distance,pscore
FID,,,,,
0,354.609,78.1,278.497,1.456,0.665823
6,353.405,0.5,724.116,1.226,0.061876
9,359.085,177.4,812.286,4.351,0.000742
14,334.235,0.0,649.112,7.048,0.205905
16,353.175,172.4,192.455,3.435,0.160527


In [71]:
subset['FID'].unique()[:5]

array([87, 73, 88, 74, 59])

In [72]:
df = subset[subset['FID'] == 9]
df.head()

,FID,Shot_angle,Bearing,Timestamp(,POI_distance,pscore
LINEARID,,,,,,
110740795643,9,359.085,177.4,812.286,4.351,0.999892
110740795643,9,359.085,177.9,813.300,7.542,0.999947
110740795643,9,359.085,359.4,845.709,6.598,0.002749
110740795643,9,359.085,359.0,846.719,10.892,0.000742


In [73]:
df.sort_values('POI_distance').iloc[0]['Timestamp(']

812.2859999999998

In [82]:
def snapshots(videopath, df,
              pscoremin = .15, # parallelism score
             anglecorrection = 5, #The camera may not be perfectly in line with the car
              sidecorrection = 3, #To bring focus more on one side
             ):
    
    vidcap = cv2.VideoCapture(videopath)
    success, image = vidcap.read()
    
    
    
    # Subset creation
    subset = df[['FID', 'Shot_angle', 'Bearing', 'Timestamp(', 'POI_distance']]
    
    #Camera bearing corection
    subset['Bearing'] = subset['Bearing']-anglecorrection
    
    
    #Pscore cleaning
    subset['pscore'] = subset.apply(parallel_scoring, axis=1, )
    #print(subset['pscore'])
    subset = subset[subset['pscore'] < pscoremin]
    
    
    

    for uid in subset['FID'].unique()[:]: #.iloc[[15, 12, 10, 37]]
        df = subset[subset['FID'] == uid]
        frame_no = df.sort_values('POI_distance').iloc[0]['Timestamp(']*1000 #MILLI SECONDS
        #print(int(frame_no)/1000)

        # CAPTURE EXTRACTION
        vidcap.set(cv2.CAP_PROP_POS_MSEC,frame_no)
        success, image = vidcap.read()
        #cv2.imwrite("../exports/{} - {} - frame.jpg".format(uid, int(frame_no)), image)

        #image = img_correction(image, house['j_delorme_heading'])

        housepic = camera(image, theta=65, phi=-90 + anglecorrection-sidecorrection, windowsize=(40, 40))
        cv2.imwrite("../exports/{} - {} - intersect.jpg".format(uid, int(frame_no)), housepic)

        print('uid:{} Done'.format(uid))
        print()
            
        
    
    print('done')
    
snapshots('../video/FLY00201.MP4', pois) #[pois['FID'] == 101]

/home/student/anaconda3/envs/tensor/lib/python3.5/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/home/student/anaconda3/envs/tensor/lib/python3.5/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


uid:87 Done

uid:77 Done

uid:85 Done

uid:61 Done

uid:79 Done

uid:75 Done

uid:65 Done

uid:63 Done

uid:93 Done

uid:83 Done

uid:95 Done

uid:17 Done

uid:69 Done

uid:37 Done

uid:0 Done

uid:71 Done

uid:81 Done

uid:99 Done

uid:24 Done

uid:22 Done

uid:34 Done

uid:28 Done

uid:32 Done

uid:102 Done

uid:101 Done

uid:26 Done

uid:15 Done

uid:20 Done

uid:7 Done

uid:19 Done

uid:18 Done

uid:54 Done

uid:11 Done

uid:52 Done

uid:53 Done

done
